In [ ]:
import sys
import time
import numpy as np
from pathlib import Path
from tqdm import tqdm
from typing import Callable, Dict, List

# pytket imports
from pytket import Circuit, OpType
from pytket.passes import DecomposeBoxes, AutoRebase#, FullPeepholeOptimise

sys.path.append("../../../tket2-rewriting/tket2_rewriting/tket2_rewriting")

# from cx_gadget.cx_gadget import CXGadget
# from zzphase_flip.zzphase_flip import ZZPhaseFlip
from three_qubit_squash.three_qubit_squash_gadget import Squash3Gadget
# from label.label_gadget import LabelGadget
# from KAK.kak_gadget import KAKGadget
# from tket.circuit import Tk2Circuit
# from tket.optimiser import BadgerOptimiser

# from circuit_generation.cx_gadget_random import CXGadgetBenchmarkCircuit
# from circuit_generation.cx_gadget_vanilla import CXGadgetVanillaBenchmarkCircuit
# from circuit_generation.cx_many_rz import CXManyRzBenchmarkCircuit
# from circuit_generation.gadget import (
#     SquashGadgetTestCircuit,
#     LabelFriendlyCircuit,
#     SimpleLabelFriendlyCircuit,
#     SimpleLabelFriendlyCircuit1,
#     KAKFriendlyCircuit,
# )

TRACE = False

In [2]:
from tket.optimiser import BadgerOptimiser
from pytket import Circuit
from pytket.circuit.display import render_circuit_jupyter
from pytket.passes import ThreeQubitSquash

opt = BadgerOptimiser(
    [
       Squash3Gadget(),
    ]
)

c1 = Circuit(3).CX(0,1).CX(1,0).CX(0,1).CX(1,0).CX(0,1).CX(1,0).CX(0,2)
c2 = Circuit(2).CX(0,1).CX(1,0).X(0).CX(0,1).CX(1,0).CX(0,1).CX(1,0)# Form Φ+ on (0,1):  H(0); CX(0,1)
c3 = Circuit(3).CX(0,1).CX(1,2).CX(1,2).CX(0,1)

opt_c1 = opt.optimise(c1)
opt_c2 = opt.optimise(c2)
opt_c3 = opt.optimise(c3)

print(f"2Q: {c1.n_2qb_gates()} → {opt_c1.n_2qb_gates()}")
print(f"2Q: {c2.n_2qb_gates()} → {opt_c2.n_2qb_gates()}")
print(f"2Q: {c3.n_2qb_gates()} → {opt_c3.n_2qb_gates()}")

# --- vs ---

print("VS")

d1 = Circuit(3).CX(0,1).CX(1,0).CX(0,1).CX(1,0).CX(0,1).CX(1,0).CX(0,2)
d2 = Circuit(2).CX(0,1).CX(1,0).X(0).CX(0,1).CX(1,0).CX(0,1).CX(1,0)
d3 = Circuit(3).CX(0,1).CX(1,2).CX(1,2).CX(0,1)

ThreeQubitSquash(allow_swaps=False).apply(c1)
ThreeQubitSquash(allow_swaps=False).apply(c2)
ThreeQubitSquash(allow_swaps=False).apply(c3)

print(f"2Q: {d1.n_2qb_gates()} → {c1.n_2qb_gates()}")
print(f"2Q: {d2.n_2qb_gates()} → {c2.n_2qb_gates()}")
print(f"2Q: {d3.n_2qb_gates()} → {c3.n_2qb_gates()}")

[3QS] tick: op=CX qs=[0, 1] arity=2 state=None
[3QS] seed: first CX on (0,1); open window_n=10
[3QS] tick: op=CX qs=[1, 0] arity=2 state=('win', 0, 1, None, 10, False, False, False)
[3QS] early-stop: same-pair CX while c is None (2Q region) -> STOP
[3QS] tick: op=CX qs=[1, 0] arity=2 state=None
[3QS] seed: first CX on (1,0); open window_n=10
[3QS] tick: op=CX qs=[0, 1] arity=2 state=('win', 1, 0, None, 10, False, False, False)
[3QS] early-stop: same-pair CX while c is None (2Q region) -> STOP
[3QS] tick: op=CX qs=[0, 1] arity=2 state=None
[3QS] seed: first CX on (0,1); open window_n=10
[3QS] tick: op=CX qs=[1, 0] arity=2 state=('win', 0, 1, None, 10, False, False, False)
[3QS] early-stop: same-pair CX while c is None (2Q region) -> STOP
[3QS] tick: op=CX qs=[1, 0] arity=2 state=None
[3QS] seed: first CX on (1,0); open window_n=10
[3QS] tick: op=CX qs=[0, 1] arity=2 state=('win', 1, 0, None, 10, False, False, False)
[3QS] early-stop: same-pair CX while c is None (2Q region) -> STOP
[3QS

In [3]:
# c = Circuit(3).H(0).CX(0,1)
# c.Z(0)   # Φ+ -> Φ- (sign flip)
# c.X(1)   # Φ- -> Ψ- (family flip)
# c.Y(0)   # Ψ- -> Ψ+ (family + sign flip)
# c.CX(2,0)   # should break Bell(0,1) labels
# c.X(0).Z(1)

In [4]:
# pip install -U qiskit pytket pytket-qiskit
# pip install pytket-quantinuum pytket-qir 
# pip install pytket-qiskit

In [ ]:
from tket.optimiser import BadgerOptimiser
from pytket import Circuit
from tket.matcher import MatchReplaceRewriter
from qiskit_pass.qiskit_gadget import QiskitGadget
from qiskit.transpiler import PassManager
from qiskit.transpiler.passes import InverseCancellation

# from tket2_rewriting.qiskit import QiskitPassManagerReplacer

opt = BadgerOptimiser(
    [
       QiskitGadget(),
    ]
)

circuit = Circuit(3).Z(2).CX(0,1).Z(0).Z(0).CX(1,2).CX(1,2).CX(0,1).CX(1,2)
circuit_opt = opt.optimise(circuit)

print(f"2Q: {circuit.n_2qb_gates()} → {circuit_opt.n_2qb_gates()}")

2Q: 5 → 1
